# Defrag Deactivation
Detects the deactivation of the Scheduled defragmentation task as seen by Slingshot APT group

## Rule Content
```
- action: global
  title: Defrag Deactivation
  id: 958d81aa-8566-4cea-a565-59ccd4df27b0
  author: Florian Roth
  description: Detects the deactivation of the Scheduled defragmentation task as seen
    by Slingshot APT group
  references:
  - https://securelist.com/apt-slingshot/84312/
  tags:
  - attack.persistence
  - attack.t1053
  - attack.s0111
  detection:
    condition: 1 of them
  falsepositives:
  - Unknown
  level: medium
- logsource:
    category: process_creation
    product: windows
  detection:
    selection1:
      CommandLine:
      - '*schtasks* /delete *Defrag\ScheduledDefrag*'
- logsource:
    product: windows
    service: security
    definition: 'Requirements: Audit Policy : Audit Other Object Access Events > Success'
  detection:
    selection2:
      EventID: 4701
      TaskName: \Microsoft\Windows\Defrag\ScheduledDefrag

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='process_command_line.keyword:(*schtasks*\ \/delete\ *Defrag\\ScheduledDefrag*)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

In [ ]:
s = searchContext.query('query_string', query='(event_id:"4701" AND task_name:"\\Microsoft\\Windows\\Defrag\\ScheduledDefrag")')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()